# Exploration of the pictures dataset

Dans ce notebook sur l'exploration du dataset d'images d'entraînement, on va étudier les principales caractéristiques de ce dernier : valeurs manquantes, qualité des images, taille, etc.


In [1]:
from pathlib import Path
from typing import Literal
import concurrent
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import numpy as np


# 1 - Build dataframe from basic image features

Comme les images sont incluses dans un dossier, on n'a pas directement de structure sous forme de dataframe ou autre. On va donc écrire une fonction qui permet de parcourir l'ensemble des images une par une et construit un dataframe à partir d'informations sur les images. Par ailleurs, on va paralléliser l'extraction d'informations pour réduire le temps de traitement.


In [3]:
n_cores = 8  # set the number of available CPU cores

img_dir = Path("../data/raw/image_data/")  # the folder of the dataset
img_names = [img for img in list(img_dir.iterdir())[:100]] # get the names of every images in the folder


# processing for each image
def process_image(img_file, mode: Literal["extract", "compute"] = "extract"):
    try:
        img = cv2.imread(str(img_file), cv2.IMREAD_UNCHANGED)
        if img is None:
            return None
        np_img = img.astype(np.float64)
        if mode == "extract":
            return {"img_id": img_file.stem,
                    "img_array": np_img}
        else:
            h, w = img.shape[:-1]
            avg, std = np.mean(np_img, axis=(0, 1)), np.std(np_img, axis=(0, 1))
        return {"img_id": img_file.stem,
                "width": w,
                "height": h,
                "memsize": img.nbytes,
                "avg_red_pixel": float(avg[0]),
                "stddev_red_pixel": float(std[0]),
                "avg_green_pixel": float(avg[1]),
                "stddev_green_pixel": float(std[1]),
                "avg_blue_pixel": float(avg[2]),
                "stddev_blue_pixel": float(std[2]),
                "avg_combined_pixel": float(np.mean(avg)),
                "stddev_combined_pixel": float(np.mean(std))}
    except Exception:
        return None

# loop for all images
def make_dataframe(img_names, mode: Literal["extract", "compute"] = "extract") -> tuple[pd.DataFrame, list]:
    data = []
    errors = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(lambda x: process_image(x, mode), img): img for img in img_names}
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            result = future.result()
            img_file = futures[future]
            if result is not None:
                data.append(result)
            else:
                errors.append(img_file)
            print(f"\rProcessing images - {100 * (i+1) / len(img_names):.2f}%   ", end='')
    df = pd.DataFrame(data)
    print()
    return df, errors


In [4]:
# INFO: this cell may take a while to complete because of the number of images to process (CPU Intel i7 11700K : ~4min with 8 cores)
df, errors = make_dataframe(img_names)  # create the dataframe


Processing images - 100.00%   


In [5]:
print(errors)  # check if some image processing gone wrong


[]


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   img_id     100 non-null    object
 1   img_array  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [7]:
df.head()


,img_id,img_array
0,image_1172460449_product_185143847,"[[[253.0, 253.0, 253.0], [253.0, 253.0, 253.0]..."
1,image_1313553701_product_4199252811,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]..."
2,image_1008107210_product_435919430,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]..."
3,image_963713094_product_19862570,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]..."
4,image_1190251689_product_2738644573,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]..."


# 2 - Exploration

Explorons ce nouveau dataset. Commençons par vérifier que toutes les images sont définies sur les $3$ canaux standards RGB.


In [ ]:
df.isna().sum()


Aucune valeur manquante n'est présente dans le dataframe, donc toutes les images sont définies en RGB.

Etudions maintenant la dimension des images.


In [ ]:
df[['width', 'height', 'memsize']].value_counts()


Toutes les images ont la même dimension, et donc la même taille en mémoire. C'est une information qui nous épargne d'effectuer une uniformisation des tailles des images. On a aussi un contrôle sur la taille mémoire des images, donc on sait que chaque image devrait avoir une durée de manipulation (pré-traitement, entraînement des modèles, etc) *a priori* identique. Comme on a vérifié ces infos, on peut les retirer du dataframe.


In [ ]:
df = df.drop(columns=['width', 'height', 'memsize'])


Notons que la taille des images amène une représentation matricielle de dimension $(500, 500, 3)$, c'est-à-dire que chaque image transmis à un modèle est une donnée de dimension $750\,000$. On est donc sur un problème de très grande dimension, nécessitant l'utilisation de modèles particulièrement bien adapté à ce genre de données. Potentiellement, on pourra considérer des techniques de réduction de dimension pour simplifier l'espace des données. On peut penser notamment à une réduction des canaux (passage RGB -> nuances de gris), des features selection par auto-encodeurs, ACP, etc.

Etudions plus en détail la répartition des pixels moyens sur l'ensemble du dataset.


In [ ]:
fig, ax = plt.subplot_mosaic([[1, 2, 3], [4, 4, 4]], figsize=(15, 10))

sns.histplot(df['avg_red_pixel'], bins=50, ax=ax[1], color='gray', zorder=2)
ax[1].set_ylabel('Counts')
xa = ax[1].twinx()
sns.kdeplot(df['avg_red_pixel'], ax=xa, color='black', zorder=2)
ax[1].set_xlabel('Values')
ax[1].set_xlim(right=260)
ax[1].grid(axis='y', linestyle='--')
ax[1].set_title('Distribution of average red channel intensity')

sns.histplot(df['avg_green_pixel'], bins=50, ax=ax[2], color='gray', zorder=2)
ax[2].set_ylabel('Counts')
xa = ax[2].twinx()
sns.kdeplot(df['avg_green_pixel'], ax=xa, color='black', zorder=2)
ax[2].set_xlabel('Values')
ax[2].set_xlim(right=260)
ax[2].grid(axis='y', linestyle='--')
ax[2].set_title('Distribution of average green channel intensity')

sns.histplot(df['avg_blue_pixel'], bins=50, ax=ax[3], color='gray', zorder=2)
ax[3].set_ylabel('Counts')
xa = ax[3].twinx()
sns.kdeplot(df['avg_blue_pixel'], ax=xa, color='black', zorder=2)
ax[3].set_xlabel('Values')
ax[3].set_xlim(right=260)
ax[3].grid(axis='y', linestyle='--')
ax[3].set_title('Distribution of average blue channel intensity')

sns.histplot(df['avg_combined_pixel'], bins=50, ax=ax[4], color='gray', zorder=2)
ax[4].set_ylabel('Counts')
xa = ax[4].twinx()
sns.kdeplot(df['avg_combined_pixel'], ax=xa, color='black', zorder=2)
ax[4].set_xlabel('Values')
ax[4].set_xlim(right=260)
ax[4].grid(axis='y', linestyle='--')
ax[4].set_title('Distribution of average pixel intensity')

plt.tight_layout()
plt.plot()


Aucune tendance particulière n'est visible sur les distributions. On peut confirmer le fait que dans l'ensemble, les images ont une intensité plutôt bonne (valeur du pixel moyen > $100$ dans la majorité des cas). On va rattacher les informations obtenues sur les images aux classes dans le but de déterminer si certaines classes ont certaines couleurs qui seraient prédominantes. On va importer les datasets correspondant et joindre le dataframe ci-dessus.


In [8]:
df_text = pd.read_csv('../data/raw/X_data.csv', index_col=0)[['productid', 'imageid']].copy(deep=True)

labels = pd.read_csv('../data/raw/Y_data.csv', index_col=0).rename(columns={'prdtypecode': 'labels'})

df_text = pd.concat([df_text, labels], axis=1)

df['productid'] = df['img_id'].apply(lambda s: s.split('_')[-1]).astype('int64')
df['imageid'] = df['img_id'].apply(lambda s: s.split('_')[1]).astype('int64')

df = df.merge(right=df_text, how='left', on=['productid', 'imageid'])  # merging the dataframes

df = df.drop(columns=['img_id'])

df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   img_array  100 non-null    object
 1   productid  100 non-null    int64 
 2   imageid    100 non-null    int64 
 3   labels     100 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 3.3+ KB


,img_array,productid,imageid,labels
0,"[[[253.0, 253.0, 253.0], [253.0, 253.0, 253.0]...",185143847,1172460449,1560
1,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",4199252811,1313553701,40
2,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",435919430,1008107210,2280
3,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",19862570,963713094,10
4,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",2738644573,1190251689,2583


Maintenant que l'on dispose des classes associées aux images, on va pouvoir faire des groupements dessus. Observons la distribution des valeurs des pixels moyen par canal et par groupe. Cela pourrait permettre de déterminer si il est nécessaire de conserver des images en RGB où si la coloration est insensible aux différentes classes.


In [31]:
df["canal_1"] = df["img_array"].apply(lambda arr: arr[:, :, 0].sum() / arr.sum())
df["canal_2"] = df["img_array"].apply(lambda arr: arr[:, :, 1].sum() / arr.sum())
df["canal_3"] = df["img_array"].apply(lambda arr: arr[:, :, 2].sum() / arr.sum())


labels_group = df[["labels", "canal_1", "canal_2", "canal_3"]].groupby("labels").agg({"canal_1": "mean",
                                                                                      "canal_2": "mean",
                                                                                      "canal_3": "mean"}, axis=1).reset_index()

labels_group.info()
labels_group.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   labels   24 non-null     int64  
 1   canal_1  24 non-null     float64
 2   canal_2  24 non-null     float64
 3   canal_3  24 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 900.0 bytes


,labels,canal_1,canal_2,canal_3
0,10,0.328909,0.334082,0.337009
1,40,0.328616,0.334099,0.337285
2,60,0.337034,0.326744,0.336222
3,1140,0.326547,0.332207,0.341246
4,1160,0.321697,0.333048,0.345255


In [ ]:
labels_group = df[['labels', 'avg_red_pixel', 'avg_green_pixel', 'avg_blue_pixel']].groupby('labels').agg('mean').reset_index()  # grouping by labels and aggregate with mean

# ratio of each channels for each label
labels_group['red_ratio'] = labels_group['avg_red_pixel'] / (labels_group['avg_red_pixel'] + labels_group['avg_green_pixel'] + labels_group['avg_blue_pixel'])
labels_group['green_ratio'] = labels_group['avg_green_pixel'] / (labels_group['avg_red_pixel'] + labels_group['avg_green_pixel'] + labels_group['avg_blue_pixel'])
labels_group['blue_ratio'] = labels_group['avg_blue_pixel'] / (labels_group['avg_red_pixel'] + labels_group['avg_green_pixel'] + labels_group['avg_blue_pixel'])

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8))
positions = range(27)

ax.bar(positions, labels_group['red_ratio'], color='darkred', tick_label=labels_group['labels'], zorder=2)
ax.bar(positions, labels_group['green_ratio'], bottom=labels_group['red_ratio'], color='darkgreen', tick_label=labels_group['labels'], zorder=2)
ax.bar(positions, labels_group['blue_ratio'], bottom=labels_group['red_ratio'] + labels_group['green_ratio'], tick_label=labels_group['labels'], zorder=2)

ax.set_xlabel('Labels')
ax.set_ylabel('Predominance of channels')
ax.set_title('Distribution of channel predominance by labels')
ax.grid(axis='y', linestyle='--')

plt.tight_layout()
plt.show()


A la vue de l'histogramme de prédominance ci-dessus, on ne peut pas conclure sur une caractérisation colorimétrique des différentes classes. On pourra donc considérer une transformation des images en nuances de gris, par exemple, pour réduire le poids des données (dimensions, taille mémoire, etc).


## 3 - Conclusion

On a vu dans ce notebook plusieurs éléments remarquables concernant le dataset d'images. Entre autres :
- Les images sont sous format RGB et standardisées en taille, leur dimension est $(500, 500, 3)$.
- Les modèles utilisés doivent donc supporter des données de très grande dimension.
- Les canaux RGB des images ne semblent pas caractériser les classes à prédire, on peut donc penser *a priori* à une réduction de dimension en transformant en nuances de gris les images -> dimension divisé par $3$.


## REMARQUE

*Extraction des valeurs de pixel en faisant la moyenne sur toute l'image, donc possiblement trop aggrégé d'où les résultats insensibles aux classes. Peut-être extraire une sous-image plutôt que juste la valeur moyenne du pixel par canal ?*
